<a href="https://colab.research.google.com/github/RuYaV/ITHub_practis/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22IThub_ML_%D1%81%D0%B5%D0%B3%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%D1%86%D0%B8%D1%8F_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%B2%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymorphy3 # установка лемматизатора pymorphy2
!pip install gensim # установка библиотеки для word2vec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 47.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
import re # Библиотека для работы со строками
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Embedding, Input, concatenate, Activation, MaxPooling1D, Conv1D, BatchNormalization, Dropout, Conv1DTranspose, Lambda, Conv2DTranspose
from tensorflow.keras import backend as K # Для создания собственной функции ошибки
from tensorflow.keras.optimizers import Adam, Adadelta # Оптимизаторы
from tensorflow.keras import utils # Для работы с категориальными данными
from google.colab import files # Для работы с файлами
import matplotlib.pyplot as plt # Для отрисовки графиков
from gensim.models import word2vec # gensim
import os # Для работы с файловой системой
#import pandas as pd # И мпортируем для работы с DataFrame
import pymorphy3 # Для лемматизации
import time # Для подсчета времени
import copy # Библиотека для копирования объектов

In [3]:
# Еще один способ загрузки датасета в Colab
!wget https://dataset.role.ru/apartment-descriptions.zip

--2025-12-17 10:34:59--  https://dataset.role.ru/apartment-descriptions.zip
Resolving dataset.role.ru (dataset.role.ru)... 176.115.205.154
Connecting to dataset.role.ru (dataset.role.ru)|176.115.205.154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66144 (65K) [application/zip]
Saving to: ‘apartment-descriptions.zip’

apartment-descripti 100%[===================>]  64.59K   382KB/s    in 0.2s    

2025-12-17 10:35:01 (382 KB/s) - ‘apartment-descriptions.zip’ saved [66144/66144]



# Функции для обработки текстов

Преобразуем тесты из описаний квартир в списке слов

In [4]:
# Чтение файла текста из файла, очитска от знаков препинания
def readText(fileName):
  f = open(fileName, 'r')
  text = f.read()
  # Определяем символы для удаления
  delSymbols = ['\n', "\t", "\ufeff", ".", "_", "-", ",", "!", "?", "–", "(", ")", "«", "»", "№", ";"]

  for dS in delSymbols: # Каждый символ в списке символов для удаления
    text = text.replace(dS, " ") # Удаляем, заменяя на пробел

  # Ищем шаблоны и символы в строке и меняем на указанную подстроку
  text = re.sub("[.]", " ", text)
  text = re.sub(":", " ", text)
  text = re.sub("<", " <", text)
  text = re.sub(">", "> ", text)

  # Сначала разделим текст на слова с помощью split, затем объединяем текст с разделителем - пробел
  # Это позволит избавиться от множественных пробелов
  text = ' '.join(text.split())
  text = text.lower() # Переводим все слова в нижний регистр
  return text # Возвращаем тексты

morph = pymorphy3.MorphAnalyzer()

# Преобразование исходного текста в список из слов в нормальной форме
def text2Words(text):
  words = text.split(' ') # Разделяем текст на пробелы
  words = [morph.parse(word)[0].normal_form for word in words] # Приводим каждое слово к нормальной форме
  return words # Возвращаем слова


# Чтение данных

In [8]:
directory = '/content/apartment-descriptions/' # Путь к папке с описаниями квартир
os.listdir(directory)[0:5] # Выведем названия 5-ти объектов

['1-комн. апартаменты, 45,2 м2 - ЖК Технопарк, Секция 2.txt',
 '2-комн. квартира, 70 м2 - ЖК Ясный, Дом 65к3.txt',
 'Апартаменты, 42,8 м2 - Комплекс Смольная 44, Корпус 1.txt',
 '4-комн. квартира, 121,90 м2 - ЖК Архитектор, Корпус 1.txt',
 '2-комн. квартира, 122,4 м2 - ЖК GRAND DELUXE на Плющихе, вл4.txt']

In [9]:
print('Итого:', len(os.listdir(directory)), 'описаний квартир.')

Итого: 47 описаний квартир.


In [10]:
apartments = [] # Список, в который запишем все наши описания
for filename in os.listdir(directory): # Проходим по всем файлам в директории
  txt = readText(directory + filename) # Читаем текст описания квартиры
  if txt != '': # Если текст не пустой, добавляем в список
    apartments.append(readText(directory + filename)) # Преобразуем файл в одну строку

In [11]:
n = 4
print(os.listdir(directory)[n]) # Выведем название n-ого описания
apartments[n] # Текст n-го описания

2-комн. квартира, 122,4 м2 - ЖК GRAND DELUXE на Плющихе, вл4.txt


'ипотека от 0 1% от банка втб первоначальный взнос 25% последняя возможность присоединиться к закрытому клубу владельцев роскошного дома по всем вопросам звоните в офис продаж мы вам все подробно расскажем <s4> просторная </s4> <s1> 2 комн квартира </s1> <s2> в жк "grand deluxe на плющихе" </s2> <s1> на 2 этаже квартира без отделки полностью готова к ремонту общая площадь 122 4 кв м жилая 41 8 кв м площадь просторной кухни столовой 32 2 кв м комнаты изолированные все окна выходят на одну сторону в квартире два совмещенных и один раздельный санузел высота потолков 3 20 м просторная и светлая 2 х комнатная квартира витражное остекление и удачная ориентация окон на юго запад </s1> удобная планировка с просторной зоной кухни позволяет рационально использовать внутреннее пространство квартиры две приватные спальни и <s1> кухня гостиная более 30 кв м </s1> квартира полностью готова к ремонту и чистовой отделке все коммуникации заведены в квартиру выполнена разводка теплоснабжения установлены

In [12]:
words = [] # Здесь будут храниться все тексты в виде списка слов
curTime = time.time() # Засечем текущее время
for i in range(len(apartments)): # Проходимся по всем файлам
  words.append(text2Words(apartments[i])) # Преобразуем очередное описание квартиры в список слов и добавляем в words
print('Преобразование заняло:', round(time.time() - curTime, 2), 'с.')

Преобразование заняло: 1.08 с.


In [13]:
wordsToTest = words[-5:] # Возьмем 5 текстов для финальной проверки обученной нейронной сети
words = words[:-5] # Для обученающей и проверочной выборок возьмем все тексты, кроме последних 10

In [14]:
print('Колличество текстов в выборке для финальной проверки в конце ноутбука:', len(wordsToTest))
print('Колличество текстов, взятых для обучающей и тестовой выборки:',len(words))
print('Колличество слов в первом описании:', len(words[1])) # Количество слов в первом договоре

Колличество текстов в выборке для финальной проверки в конце ноутбука: 5
Колличество текстов, взятых для обучающей и тестовой выборки: 42
Колличество слов в первом описании: 151


In [15]:
print('Пример исходного текста:')
print(apartments[4], '\n')
print('Тот же текст, представленный в виде списка из слов в начальной форме:')
print(words[4])

Пример исходного текста:
ипотека от 0 1% от банка втб первоначальный взнос 25% последняя возможность присоединиться к закрытому клубу владельцев роскошного дома по всем вопросам звоните в офис продаж мы вам все подробно расскажем <s4> просторная </s4> <s1> 2 комн квартира </s1> <s2> в жк "grand deluxe на плющихе" </s2> <s1> на 2 этаже квартира без отделки полностью готова к ремонту общая площадь 122 4 кв м жилая 41 8 кв м площадь просторной кухни столовой 32 2 кв м комнаты изолированные все окна выходят на одну сторону в квартире два совмещенных и один раздельный санузел высота потолков 3 20 м просторная и светлая 2 х комнатная квартира витражное остекление и удачная ориентация окон на юго запад </s1> удобная планировка с просторной зоной кухни позволяет рационально использовать внутреннее пространство квартиры две приватные спальни и <s1> кухня гостиная более 30 кв м </s1> квартира полностью готова к ремонту и чистовой отделке все коммуникации заведены в квартиру выполнена разводка те

# Преобразование текста и формирование xTrain

In [16]:
# lower=True - приводим слова к нижнему регистру
# char_level=False - просим токенайзер учитывать слова, а не отдельные символы
tokenizer = Tokenizer(lower=True, filters = '', char_level=False)

tokenizer.fit_on_texts(words) # Формируем словарь частотности

# Преобразуем из массива в словарь
clean_voc = {}
for item in tokenizer.word_index.items():
  clean_voc[item[0]] = item[1]

In [17]:
print('Словарь всех слов по частоте:')
print(clean_voc, '\n')
print('Длинна словаря:', len(clean_voc))

tag = '<s1>'
print('Индекс тега', tag, ':' ,clean_voc[tag])

Словарь всех слов по частоте:
{'в': 1, 'и': 2, 'на': 3, '<s2>': 4, '</s2>': 5, 'квартира': 6, '<s1>': 7, '</s1>': 8, 'с': 9, 'м': 10, 'от': 11, 'этаж': 12, 'дом': 13, '<s3>': 14, '</s3>': 15, 'комплекс': 16, '1': 17, 'площадь': 18, '2': 19, 'кв': 20, 'по': 21, 'для': 22, 'москва': 23, 'апартамент': 24, 'жилой': 25, 'проект': 26, 'из': 27, '<s4>': 28, '</s4>': 29, 'район': 30, '5': 31, 'жк': 32, 'общий': 33, '3': 34, 'до': 35, 'расположить': 36, 'отделка': 37, 'корпус': 38, 'высота': 39, 'окно': 40, 'квартал': 41, 'метро': 42, 'продаваться': 43, 'минута': 44, 'зона': 45, 'это': 46, 'детский': 47, 'всё': 48, 'застройщик': 49, 'который': 50, 'станция': 51, 'центр': 52, 'потолок': 53, 'один': 54, 'км': 55, 'современный': 56, 'кухня': 57, 'находиться': 58, 'просторный': 59, 'компания': 60, 'планировка': 61, 'к': 62, 'подземный': 63, 'паркинг': 64, 'система': 65, 'более': 66, 'два': 67, 'лифт': 68, 'год': 69, 'весь': 70, 'территория': 71, '4': 72, 'площадка': 73, 'быть': 74, 'парк': 75, '8':

In [18]:
# Преобразовываем текст в последовательность индексов согласно частотному словарю
tok_agreem = tokenizer.texts_to_sequences(words) # Обучающие тесты в индексы

print("Взглянем на фрагмент обучающего текста:")
print("В виде оригинального текста:              ", words[4][:20])
print("Он же в виде последовательности индексов: ", tok_agreem[4][:20], '\n')

Взглянем на фрагмент обучающего текста:
В виде оригинального текста:               ['ипотека', 'от', '0', '1%', 'от', 'банк', 'втб', 'первоначальный', 'взнос', '25%', 'последний', 'возможность', 'присоединиться', 'к', 'закрытый', 'клуб', 'владелец', 'роскошный', 'дом', 'по']
Он же в виде последовательности индексов:  [178, 11, 185, 904, 11, 82, 589, 905, 906, 907, 590, 152, 908, 62, 210, 273, 276, 591, 13, 21] 



# Формирование yTrain

Алгоритм:

Каждому индексу (слову) из договора необходимо поставить в соответсвие вектор из 4-х элементов, соответствующих тегам категорий

In [19]:
num_tags = 4

# s1 - о квартире
# s2 - о ЖК
# s3 - расположение
# s4 - цена

In [20]:
def getTagsIndex(voc, num_tags):
  openTags = {}
  closeTags = {}

  # Формируем список открывающих и закрывающих тегов исходя из того, сколько их у нас
  for c in range(num_tags):
    openTags["<s"+str(c+1)+">"] = voc["<s"+str(c+1)+">"]
    closeTags["</s"+str(c+1)+">"] = voc["</s"+str(c+1)+">"]

  return openTags, closeTags


def makeTrains(words, num_tags):
  xTrain = []
  yTrain = []

  openTags = {}
  closeTags = {}

  for c in range(num_tags):
    openTags["<s"+str(c+1)+">"] = c
    closeTags["</s"+str(c+1)+">"] = c

  # Формируем Y, заполняя его нулями. Нужно столько нулей, сколько у нас классов
  y = []
  for i in range(num_tags):
    y.append(0) # То есть в конце тут будет массив [0, 0, 0, 0]

  # Перебираем все слова
  for w in words:
    # Если слово является открывающим тегом, то "взводим" единичку в нужной позиции
    if w in openTags:
      y[openTags[w]] = 1
      # Если слово является закрывающим тегом, то обнуляем значение Y в нужной позиции
    elif w in closeTags:
      y[closeTags[w]] = 0
    else:
      xTrain.append(copy.copy(w))
      yTrain.append(copy.copy(y))

  return xTrain, yTrain


# Получение списка слов из индексов (эту функцию не используем в этом ноутбуке, но она подандобится в вашей домашней работе)
def reverseIndex(clean_voc, x):
  reverse_word_map = dict(map(reversed, clean_voc.items())) # Берем пары значений всего словаря и размечаем наоборот, т.е. value:key
  words = [reverse_word_map.get(letter) for letter in x] # Вытаскиваем по каждому ключу в список
  return words # Возвращаем полученный текст

In [21]:
openTags, closeTags = getTagsIndex(clean_voc, num_tags)
print(openTags, closeTags)

{'<s1>': 7, '<s2>': 4, '<s3>': 14, '<s4>': 28} {'</s1>': 8, '</s2>': 5, '</s3>': 15, '</s4>': 29}


In [22]:
# Формируем xTrain и yTrain
xData = []
yData = []
for post in words:
  x, y = makeTrains(post, num_tags)
  xData += x
  yData += y

In [23]:
print('Длина xData:', len(xData))
print('Длина yData:', len(yData))

print('Посмотрим на исходный набор слов: ', words[0][0:80])
print('Посмотрим на часть из xData:     ', xData[0:80])
print('Посмотрим на часть из yData:     ', yData[0:80])

Длина xData: 7565
Длина yData: 7565
Посмотрим на исходный набор слов:  ['продаваться', '<s1>', 'однокомнатный', 'апартамент', 'площадь', '45', '2', 'кв', 'м', '</s1>', '<s2>', 'в', 'новостройка', 'технопарк', '</s2>', '<s3>', 'по', 'адрес', 'москва', 'юао', 'даниловский', 'район', 'нагатинский', 'набережная', '</s3>', '<s1>', 'общий', 'площадь', 'апартамент', '45', '2', 'кв', 'м', '</s1>', 'этаж', '10', 'из', '15', 'секция', '2', '<s2>', 'тип', 'проект', 'по', 'который', 'построить', 'дом', 'монолит', 'компания', 'застройщик', 'инвестиционный', 'перспектива', '</s2>', '<s4>', 'стоимость', 'апартамент', '15800000', 'р', 'дом', 'сдать', '</s4>', 'более', 'подробный', 'информация', 'по', 'телефон', '<s2>', 'технопарк', 'это', 'многофункциональный', 'комплекс', 'апартамент', 'от', 'девелопер', 'инвестиционный', 'перспектива', '</s2>', '<s3>', 'проект', 'расположить']
Посмотрим на часть из xData:      ['продаваться', 'однокомнатный', 'апартамент', 'площадь', '45', '2', 'кв', 'м', 'в', 'ново

# Разделение выборки на окна

In [24]:
# Формируем выборку из индексов
def getSetFromIndexes(wordIndexes, xLen, step):
  xBatch = [] # Массив для фрагментов текста
  wordsLen = len(wordIndexes) # Определяем длинну текста
  index = 0 # Задаем стартовый индекс

  while (index + xLen <= wordsLen): # Пока сумма индекса с длинной фрагмента меньше или равна числу слов в выборке
    xBatch.append(wordIndexes[index:index+xLen]) # Добавляем X в лист фразментов текста
    index += step # Сдвигаемся на step

  return xBatch # Лист для фрагментов текста

In [154]:
xLen = 64 # Длина окна
step = 32 # Шаг
embeddingSize = 300 # Размер эмбеддинга

In [155]:
# Генерируем наборы с заданными параметрами окна
xTrain = getSetFromIndexes(xData, xLen, step) # Последовательность из xLen слов
yTrain = getSetFromIndexes(yData, xLen, step) # Последовательность из xLen-тегов

In [156]:
print('Длинна xTrain:', len(xTrain))
print('Длинна yTrain:', len(yTrain))
print('Длинна примера из xTrain:',len(xTrain[0]))
print('Длинна примера из yTrain:',len(yTrain[0]), '\n')
print('Пример xTrain', xTrain[0])
print('Пример yTrain', yTrain[0], '\n')

Длинна xTrain: 235
Длинна yTrain: 235
Длинна примера из xTrain: 64
Длинна примера из yTrain: 64 

Пример xTrain ['продаваться', 'однокомнатный', 'апартамент', 'площадь', '45', '2', 'кв', 'м', 'в', 'новостройка', 'технопарк', 'по', 'адрес', 'москва', 'юао', 'даниловский', 'район', 'нагатинский', 'набережная', 'общий', 'площадь', 'апартамент', '45', '2', 'кв', 'м', 'этаж', '10', 'из', '15', 'секция', '2', 'тип', 'проект', 'по', 'который', 'построить', 'дом', 'монолит', 'компания', 'застройщик', 'инвестиционный', 'перспектива', 'стоимость', 'апартамент', '15800000', 'р', 'дом', 'сдать', 'более', 'подробный', 'информация', 'по', 'телефон', 'технопарк', 'это', 'многофункциональный', 'комплекс', 'апартамент', 'от', 'девелопер', 'инвестиционный', 'перспектива', 'проект']
Пример yTrain [[0, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], 

# Создание xTrain и  yTrain при помощи метода Word2wec GENSIM

In [157]:
# Создаем выборки
def getSets(model, senI, tagI):
  xVector = [] # Здесь будет лежать embedding представление каждого из индексов
  tmp = [] # Временный список
  for text in senI: # Проходимся по каждому тексту-списку
    tmp=[]
    for word in text: # Проходимся по каждому слову в тексте-списке
      tmp.append(model.wv.get_vector(word))

    xVector.append(tmp)

  return np.array(xVector), np.array(tagI)

In [158]:
# Передаем в word2vec списки списков слов для обучения
# vector_size = embeddingSize - размер эмбеддинга
# window = 10 - минимальное расстояние между словами в эмбеддинге
# min_count = 1 - игнорирование всех слов с частотой, меньше, чем 1
# workers = 10 - число потоков обучения эмбеддинга
# epochs = 10 - число эпох обучения эмбеддинга

modelGENSIM = word2vec.Word2Vec(xTrain, vector_size=embeddingSize, window=10, min_count=1, workers=10, epochs=10)

In [159]:
xTrainGENSIM, yTrainGENSIM = getSets(modelGENSIM, xTrain, yTrain)

In [160]:
print('Размерность полученного xTrain:', xTrainGENSIM.shape)
print('Размерность полученного yTrain:', yTrainGENSIM.shape)

Размерность полученного xTrain: (235, 64, 1000)
Размерность полученного yTrain: (235, 64, 4)


##**UNET**

In [161]:
from tensorflow.keras import losses

In [162]:
# Функция, которая оценивает пересечение областей. Нужна для accuracy
def dice_coef(y_true, y_pred):
  y_true = tf.cast(y_true, tf.float32)
  y_pred = tf.cast(y_pred, tf.float32)
  return 1-(2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.)

In [163]:
# Функция для создания сети UNET
def create_unet(k=1, num_classes=4, input_shape=(30, 300), af="relu"):
    print("Коэффициент:", k)
    print("Количество классов:", num_classes)
    print("input_shape:", input_shape)
    print("Функция активации:", af)

    # У кераса нет Conv1DTranspose, так что прописываем вручную. Нужен для разворота после maxpooling
    def Conv1DTranspose(input_tensor, filters, kernel_size=1, strides=2, padding='same'):
        """
        input_tensor: входной тензор (batch_size, time_steps, dims)
        filters: int, output dimension, выходной тензор будет иметь размер (batch_size, time_steps, filters)
        kernel_size: размер ядра свертки
        strides: int, шаг ядра
        padding: 'same'
        """
        x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
        x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), strides=(strides, 1), padding=padding)(x)
        x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
        return x

    img_input = Input(input_shape)

    # Block 1
    x = Conv1D(int(64 * k) , 4, padding='same')(img_input)
    x = BatchNormalization()(x)
    x = Activation(af)(x)
    x = Dropout(0.2)(x)

    x = Conv1D(int(64 * k) , 4, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(af)(x)
    x = Dropout(0.2)(x)
    block_1_out = x

    x = MaxPooling1D()(x)

    # Block 2
    x = Conv1D(int(128 * k) , 4, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(af)(x)
    x = Dropout(0.3)(x)

    x = Conv1D(int(128 * k) , 4, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(af)(x)
    x = Dropout(0.3)(x)
    block_2_out = x

    x = MaxPooling1D()(x)

    # Block 3
    x = Conv1D(int(256 * k) , 4, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(af)(x)
    x = Dropout(0.4)(x)

    x = Conv1D(int(256 * k) , 4, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(af)(x)
    x = Dropout(0.4)(x)

    x = Conv1D(int(256 * k) , 4, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(af)(x)
    x = Dropout(0.4)(x)
    block_3_out = x
    x = MaxPooling1D()(x)

    # Block 4
    x = Conv1D(int(512 * k) , 4, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(af)(x)
    x = Dropout(0.5)(x)

    x = Conv1D(int(512 * k) , 4, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(af)(x)
    x = Dropout(0.5)(x)

    x = Conv1D(int(512 * k) , 4, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(af)(x)
    x = Dropout(0.5)(x)
    block_4_out = x
    x = block_4_out

    # UP 3
    x = Conv1DTranspose(x, 256, kernel_size=2, strides=2, padding='same')
    x = BatchNormalization()(x)
    x = Activation(af)(x)
    x = Dropout(0.2)(x)

    x = concatenate([x, block_3_out])
    x = Conv1D(int(256 * k) , 4, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(af)(x)
    x = Dropout(0.2)(x)

    x = Conv1D(int(256 * k) , 4, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(af)(x)
    x = Dropout(0.2)(x)

    # UP 4
    x = Conv1DTranspose(x, 128, kernel_size=2, strides=2, padding='same')
    x = BatchNormalization()(x)
    x = Activation(af)(x)
    x = Dropout(0.3)(x)

    x = concatenate([x, block_2_out])
    x = Conv1D(int(128 * k) , 4, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(af)(x)
    x = Dropout(0.3)(x)

    # UP 5
    x = Conv1DTranspose(x, 64, kernel_size=2, strides=2, padding='same')
    x = BatchNormalization()(x)
    x = Activation(af)(x)
    x = Dropout(0.4)(x)

    x = concatenate([x, block_1_out])
    x = Conv1D(int(64 * k) , 4, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(af)(x)
    x = Dropout(0.4)(x)

    x = Conv1D(num_classes, 4, activation='softmax', padding='same')(x)

    model = Model(img_input, x)

#    model.compile(optimizer=Adam(0.001),
#                  loss='categorical_crossentropy',
#                  metrics=[dice_coef])
    model.compile(optimizer=Adam(0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Обучение нейросети

In [164]:
model = create_unet(k=0.5, num_classes=num_tags, input_shape=(xLen, embeddingSize), af="sigmoid")
history = model.fit(xTrainGENSIM, yTrainGENSIM, epochs=40, batch_size=1, validation_split=0.25)

Коэффициент: 0.5
Количество классов: 4
input_shape: (64, 1000)
Функция активации: sigmoid
Epoch 1/40
176/176 ━━━━━━━━━━━━━━━━━━━━ 28s 19ms/step - accuracy: 0.2680 - loss: 0.8094 - val_accuracy: 0.6388 - val_loss: 0.6798
Epoch 2/40
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.4220 - loss: 0.7107 - val_accuracy: 0.6343 - val_loss: 0.6715
Epoch 3/40
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.3827 - loss: 0.7008 - val_accuracy: 0.6401 - val_loss: 0.6711
Epoch 4/40
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.4299 - loss: 0.6559 - val_accuracy: 0.1933 - val_loss: 0.6840
Epoch 5/40
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3808 - loss: 0.6776 - val_accuracy: 0.1642 - val_loss: 0.6837
Epoch 6/40
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3635 - loss: 0.7054 - val_accuracy: 0.6343 - val_loss: 0.6693
Epoch 7/40
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4027 - loss: 0.7277 - val_accuracy: 0.6520 - val_loss: 0.6960
Epoch 8/40
1

In [166]:
print(max(history.history["val_accuracy"]))

0.6520127058029175


**Проверка работы сети**

In [108]:
# Преобразовываем текст в последовательность индексов согласно частотному словарю
tok_agreemTest = tokenizer.texts_to_sequences(wordsToTest) # Обучающие тесты в индексы

print("Взглянем на фрагмент тестового текста:")
print("В виде оригинального текста:              ", wordsToTest[4][:20])
print("Он же в виде последовательности индексов: ", tok_agreemTest[4][:20], '\n')

Взглянем на фрагмент тестового текста:
В виде оригинального текста:               ['артикул', '3', '26', '<s3>', 'на', 'живописный', 'стрелка', 'москва', 'река', 'и', 'сходня', '</s3>', 'продаваться', '<s1>', '3', 'к', 'апартамент', 'с', 'отделка', '"white']
Он же в виде последовательности индексов:  [34, 532, 14, 3, 398, 23, 134, 2, 15, 43, 7, 34, 62, 24, 9, 37, 18, 19, 20, 10] 



In [167]:
# Формируем xTrain и yTrain
xDataTest = []
yDataTest = []
for post in words:
  x, y = makeTrains(post, num_tags)
  xDataTest += x
  yDataTest += y

In [110]:
print('Длина xDataTest:', len(xDataTest))
print('Длина yDataTest:', len(yDataTest))

print('Посмотрим на исходный набор слов: ', wordsToTest[0][50:80])
print('Посмотрим на часть из xDataTest:     ', xDataTest[50:80])
print('Посмотрим на часть из yDataTest:     ', yDataTest[50:80])

Длина xDataTest: 7565
Длина yDataTest: 7565
Посмотрим на исходный набор слов:  ['встроить', 'кухня', 'техника', 'мебель', 'это', 'жилой', 'квартира', 'с', 'прописка', 'не', 'апартамент', '<s3>', 'удачный', 'расположение', '</s3>', 'вид', 'из', 'окно', 'этаж', 'коммунальный', 'платёж', 'в', 'среднее', '3000', 'покупать', 'без', 'ипотека', 'и', 'мат', 'капитал']
Посмотрим на часть из xDataTest:      ['подробный', 'информация', 'по', 'телефон', 'технопарк', 'это', 'многофункциональный', 'комплекс', 'апартамент', 'от', 'девелопер', 'инвестиционный', 'перспектива', 'проект', 'расположить', 'поблизости', 'от', 'москва', 'река', 'занимать', '2', '5', 'га', 'на', 'который', 'размещаться', '5', 'монолитный', '15', 'этажный']
Посмотрим на часть из yDataTest:      [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0

In [168]:
# Генерируем наборы с заданными параметрами окна
xTest = getSetFromIndexes(xDataTest, xLen, step) # Последовательность из xLen слов
yTest = getSetFromIndexes(yDataTest, xLen, step) # Последовательность из xLen-тегов

In [112]:
print('Длинна xTest:', len(xTest))
print('Длинна yTest:', len(yTest))
print('Длинна примера из xTest:',len(xTest[0]))
print('Длинна примера из yTrain:',len(yTest[0]), '\n')
print('Пример xTest', xTest[0])
print('Пример yTest', yTest[0], '\n')

print('Первый пример xTest:', xTest[0][step-5:step+5])
print('Второй пример xTest:', xTest[1][:10])

Длинна xTest: 235
Длинна yTest: 235
Длинна примера из xTest: 64
Длинна примера из yTrain: 64 

Пример xTest ['продаваться', 'однокомнатный', 'апартамент', 'площадь', '45', '2', 'кв', 'м', 'в', 'новостройка', 'технопарк', 'по', 'адрес', 'москва', 'юао', 'даниловский', 'район', 'нагатинский', 'набережная', 'общий', 'площадь', 'апартамент', '45', '2', 'кв', 'м', 'этаж', '10', 'из', '15', 'секция', '2', 'тип', 'проект', 'по', 'который', 'построить', 'дом', 'монолит', 'компания', 'застройщик', 'инвестиционный', 'перспектива', 'стоимость', 'апартамент', '15800000', 'р', 'дом', 'сдать', 'более', 'подробный', 'информация', 'по', 'телефон', 'технопарк', 'это', 'многофункциональный', 'комплекс', 'апартамент', 'от', 'девелопер', 'инвестиционный', 'перспектива', 'проект']
Пример yTest [[0, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0

In [169]:
# Передаем в word2vec списки списков слов для обучения
# vector_size = embeddingSize - размер эмбеддинга
# window = 10 - минимальное расстояние между словами в эмбеддинге
# min_count = 1 - игнорирование всех слов с частотой, меньше, чем 1
# workers = 10 - число потоков обучения эмбеддинга
# epochs = 10 - число эпох обучения эмбеддинга

modelGENSIM_T = word2vec.Word2Vec(xTest, vector_size=embeddingSize, window=10, min_count=1, workers=10, epochs=10)

In [170]:
xTestGENSIM_T, yTestGENSIM_T = getSets(modelGENSIM_T, xTest, yTest)

In [115]:
print('Размерность полученного xTrain:', xTestGENSIM_T.shape)
print('Размерность полученного yTrain:', yTestGENSIM_T.shape)

Размерность полученного xTrain: (235, 64, 300)
Размерность полученного yTrain: (235, 64, 4)


In [116]:
tags = ['S1', 'S2', 'S3', 'S4']
# s1 - о квартире
# s2 - о ЖК
# s3 - расположение
# s4 - цена

In [117]:
# Функция, выводящая точность распознавания каждой категории отдельно
def recognizeSet(tagI, pred, tags, length, value):
  total=0

  for j in range(len(tags)): # общее количество тегов
    correct=0
    for i in range(len(tagI)): # проходимся по каждому списку списка тегов
      for k in range(length): # проходимся по каждому тегу
        if tagI[i][k][j]==(pred[i][k][j]>value).astype(int): # если соответствующие индексы совпадают, значит сеть распознала верно
          correct+=1
    print("Сеть распознала категорию '{}' на {}%".format(tags[j],round(100*correct/(len(tagI)*length), 2)))
    total += 100 * correct / (len(tagI)*length)
  print("Cредняя точность {}%".format(round(total/len(tags), 2)))

In [171]:
pred = model.predict(xTestGENSIM_T) # сделаем предсказание

8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 226ms/step


#83.54% + 5% = 88.5%
ебучий барьер: 87.73%

In [172]:
recognizeSet(yTestGENSIM_T, pred, tags, xLen, 0.999)

Сеть распознала категорию 'S1' на 81.01%
Сеть распознала категорию 'S2' на 84.05%
Сеть распознала категорию 'S3' на 87.66%
Сеть распознала категорию 'S4' на 96.4%
Cредняя точность 87.28%
